<a href="https://colab.research.google.com/github/mohansameer1983/NLP/blob/main/FinalProject/PersonalAssistant_hybrid_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
import re




import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.stem import LancasterStemmer, WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path='/content/drive/MyDrive/MMAI891/FinalProject_TrainingData.csv'
path_test='/content/drive/MyDrive/MMAI891/FinalProject_TestData.csv'

In [ ]:
path_test


'/content/drive/MyDrive/MMAI891/FinalProject_TestData.csv'

In [ ]:
df = pd.read_csv(path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8884 entries, 0 to 8883
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              8884 non-null   int64  
 1   message         8884 non-null   object 
 2   label           8884 non-null   object 
 3   Unnamed: 3      0 non-null      float64
 4   Unnamed: 4      0 non-null      float64
 5   Distinct Label  64 non-null     object 
dtypes: float64(2), int64(1), object(3)
memory usage: 416.6+ KB


In [ ]:
df_test= pd.read_csv(path_test)

In [ ]:
x=df['message']
y=df['label']

In [ ]:
X_test=df_test['message']

In [ ]:
y.head()

0        iot_cleaning
1      general_negate
2    datetime_convert
3         lists_query
4         email_query
Name: label, dtype: object

encoding labels

In [ ]:
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
y = labelencoder.fit_transform(y)



# preprocessing

tokenizer

case normalization

N-grams

remove numbers

stemming


# Vectorization

In [ ]:
pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 3.1 MB/s 
     |████████████████████████████████| 4.0 MB 9.2 MB/s 
     |████████████████████████████████| 1.2 MB 34.0 MB/s 
     |████████████████████████████████| 77 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 46.3 MB/s 
     |████████████████████████████████| 895 kB 59.8 MB/s 
     |████████████████████████████████| 6.5 MB 38.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=8d23e9b41c9fc3e1db1c7d88bdec77eb2fea2cda66fb5202757c8888ec3ceab2
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')


x_embeddings = embedder.encode(x, convert_to_numpy=True)




In [ ]:
x_embeddings

array([[-0.5887814 , -0.15007058, -0.24277775, ...,  0.3264495 ,
        -0.46130696, -0.3184985 ],
       [ 0.21024798, -0.1594509 ,  0.34390777, ...,  0.08608998,
         0.76303935, -0.13470525],
       [ 1.071969  , -0.6746578 ,  0.50290245, ..., -0.03767654,
        -0.70019597,  0.12923133],
       ...,
       [-0.15600432,  0.10299429,  0.4183066 , ...,  0.6624442 ,
        -0.36820185,  0.23117273],
       [ 0.0028928 , -0.31685698,  0.5755866 , ...,  0.35636103,
        -0.00232087,  0.47726956],
       [-0.6391246 , -0.2957041 ,  0.0093051 , ...,  0.53227013,
        -0.20714207,  0.23913145]], dtype=float32)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x_embeddings, y, test_size=0.2, random_state=42)



# Build Model XGBOOST

In [ ]:
from xgboost import XGBClassifier
clf_xg = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1)
clf_xg.fit(X_train, y_train)

XGBClassifier(max_depth=5, objective='multi:softprob')

In [ ]:
y_val_pred =clf_xg.predict(X_val)

In [ ]:
from sklearn.metrics.cluster import adjusted_mutual_info_score, adjusted_rand_score

adjusted_rand = adjusted_rand_score(y_val, y_val_pred)
adjusted_mutual_info = adjusted_mutual_info_score(y_val, y_val_pred)

print("ARI: {}".format(adjusted_rand))
print("AMI: {}".format(adjusted_mutual_info))

ARI: 0.7203961078060191
AMI: 0.8039487520182359


In [ ]:
x_test_embeddings = embedder.encode(X_test, convert_to_numpy=True)

In [ ]:
y_test_pred =clf_xg.predict(x_test_embeddings)

In [ ]:
my_submission = pd.DataFrame({'Id': df_test['id'], 'label': y_test_pred})
print(my_submission.head())
my_submission.to_csv('Submission_SBERT_XGBoost.csv', index=False)

      Id  label
0  10873     51
1   8491     53
2   4549     25
3   5948     36
4   1663      4


# New Section

# New Section

# New Section